# 01_pulling_cleaning_data
A notebook to ingest various local election data, standardise it and export it. 

In order to standardise the 2022 data (not at the ward-level, we may want to consider imputing ward-level scores based on previous results?)

Election result data: 
- 2018 local election results, ward-level
- 2021 local election results, ward-level
- 2022 local election results, council-level

Census data
- 2021 top-level census insights
- 2021-2011 population change, from census

NOTE: Using Python 3.10

NL, 10/07/22

## IMPORTS

In [245]:
import pandas as pd
import numpy as np

## PATHS

Add paths to data, URLS, etc. below as required

In [16]:
DATA_PATH = '../data/'

In [67]:
WARD_RESULTS_2018_RAW = DATA_PATH+'raw/2018_LocalElectionResults_AndrewTealLeap.csv'
WARD_RESULTS_2021_RAW = DATA_PATH+'raw/2021_LocalElectionResullts_BritainElectis.xlsx'
COUNCIL_RESULTS_2022_RAW = DATA_PATH+'raw/2022_LocalElectionResults_CouncilLevel.xlsx'

In [59]:
CENSUS_2021_2011_POP_CHANGE = DATA_PATH+'raw/2021_2011_Census_population_change.xlsx'
CENSUS_2021_TOP_LEVEL_FINDINGS = DATA_PATH+'raw/2021_Census_TopLevelSummaryStats.xlsx'

## INIT / DATA IN

In [143]:
colnames_2018 = ['local_authority', 'local_authority_ons_code', 'ward', 'ward_ons_code', 'candidate_name', 'party', 'n_votes', 'cndidate_won']

In [144]:
colnames_2021_base = ['ward_ons_code', 'local_authority', 'ward', 'year_last_election', 'winner_last_election', 'winner_2021']
party_names = ['con', 'lab', 'ldem', 'ukip', 'grn', 'snp', 'pc', 'ind', 'indgrp', 'reg', 'resass', 'oth']
suffixes = ['_2021_vote', '_previous_vote', '_2021_perc', '_previous_perc']

In [145]:
party_columns = []
for suffix in suffixes:
    for party in party_names:
        tmp = party+suffix
        party_columns.append(tmp)

colnames_2021 = colnames_2021_base+party_columns

In [228]:
raw_results_2018_df = pd.read_csv(WARD_RESULTS_2018_RAW, header=0, names=colnames_2018)
raw_results_2021_df = pd.read_excel(WARD_RESULTS_2021_RAW, sheet_name='results', header=None, skiprows=2, names=colnames_2021)

Note: our 2022 results are only at the council level. For this purpose, it might make sense to collapse our other results data to the council level also.

In [71]:
colnames_2022 = [
    'ons_code', 
    'council_name', 
    'boundary_change', 
    'new_council', 
    'council_type', 
    'seats', 
    'region', 
    'con_pre', 
    'lab_pre', 
    'ldem_pre', 
    'grn_pre', 
    'oth_pre', 
    'total_pre', 
    'control_pre', 
    'con_post', 
    'lab_post', 
    'ldem_post', 
    'grn_post', 
    'oth_post', 
    'vacant_post', 
    'total_post', 
    'control_post', 
    'con_net', 
    'lab_net', 
    'ldem_net', 
    'grn_net', 
    'oth_net']

In [87]:
raw_results_2022_df = pd.read_excel(COUNCIL_RESULTS_2022_RAW, skiprows=1, names=colnames_2022)

In [90]:
raw_results_2022_df = raw_results_2022_df.replace('-', np.NaN)

For census files, it seems that we need to do data imports from row 6 of the files.

In [64]:
census_2021_11_pop_change_df = pd.read_excel(CENSUS_2021_2011_POP_CHANGE, sheet_name='Population change', skiprows=5)
census_2021_11_pop_change_df = census_2021_11_pop_change_df.rename(columns={'Area code [note 2]': 'area_code',
'Area name' : 'area_name', 'All persons, 2011' : 'n_people_2011', 'All persons, 2021' : 'n_people_2021', 'Percentage change' : 'perc_change'})

## DATA WRANGLING / TRANSFORMATION

let's create some council-level files for our 2018 and 2021 data, so as to ensure comparability. 

let's also save these files for future use. 

1. 2018. ward-level -> council-level

In [229]:
party_names_2018 = {
    'C' : 'con',
    'Lab' : 'lab',
    'LD' : 'ldem',
    'Grn' : 'grn',
    'UKIP' : 'ukip',
    'Ind' : 'ind'
}

In [230]:
# change names of main parties to our standardised format
for k,v in party_names_2018.items():
    raw_results_2018_df.loc[raw_results_2018_df['party']==k, 'party'] = v

In [231]:
# rename all others to 'other'
raw_results_2018_df.loc[~raw_results_2018_df['party'].isin(party_names_2018.values()), 'party'] = 'other'

In [232]:
tmp_df = raw_results_2018_df.groupby(['local_authority', 'party']).sum().reset_index()

In [233]:
attach_df = tmp_df.pivot(columns='party', values='n_votes').reset_index(drop=True)

In [234]:
tmp_df = pd.concat([tmp_df, attach_df], axis=1)

In [235]:
council_results_2018_df = tmp_df.groupby('local_authority').sum()
council_results_2018_df = council_results_2018_df.reset_index()

In [214]:
council_results_2018_df = council_results_2018_df[['local_authority', 'n_votes', 'con', 'grn', 'ind', 'lab', 'ldem', 'other', 'ukip']]


In [236]:
# add in the council-level ons code
council_ons_merge_points_df = raw_results_2018_df[['local_authority', 'local_authority_ons_code']].drop_duplicates().reset_index(drop=True)

In [239]:
council_results_2018_df = council_results_2018_df.merge(council_ons_merge_points_df, on='local_authority', how='left')

In [242]:
# write out this as formatted
council_results_2018_df.to_csv(DATA_PATH+'fmtd/2018_council_level_results.csv', index=False)

2. 2021 data, council level results

In [244]:
raw_results_2021_df.groupby('local_authority').sum()

,con_2021_vote,lab_2021_vote,grn_2021_vote,snp_2021_vote,pc_2021_vote,ind_2021_vote,indgrp_2021_vote,resass_2021_vote,oth_2021_vote,con_previous_vote,...,ldem_previous_perc,ukip_previous_perc,grn_previous_perc,snp_previous_perc,pc_previous_perc,ind_previous_perc,indgrp_previous_perc,reg_previous_perc,resass_previous_perc,oth_previous_perc
local_authority,,,,,,,,,,,,,,,,,,,,,
Adur,8058.0,5610.0,2436.0,0.0,0.0,1122.0,0.0,0.0,0.0,5729.0,...,100.769288,324.029660,43.546296,0.0,0.0,82.278779,0.00000,0.0,0.000000,0.000000
Allerdale,2390.0,1729.0,116.0,0.0,0.0,1465.0,0.0,0.0,33.0,2025.0,...,0.000000,0.000000,33.127226,0.0,0.0,93.639423,34.95935,0.0,0.000000,0.000000
Amber Valley,14244.0,9218.0,2077.0,0.0,0.0,0.0,0.0,0.0,18.0,8418.0,...,35.864406,260.148826,65.425321,0.0,0.0,7.201759,0.00000,0.0,0.000000,2.245024
Arun,1037.0,292.0,93.0,0.0,0.0,132.0,0.0,0.0,45.0,975.0,...,90.995785,0.000000,0.000000,0.0,0.0,31.933439,0.00000,0.0,0.000000,0.000000
Ashfield,426.0,857.0,0.0,0.0,0.0,0.0,2767.0,0.0,0.0,206.0,...,0.000000,7.109915,0.000000,0.0,0.0,153.754376,0.00000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wolverhampton,24624.0,28700.0,3609.0,0.0,0.0,217.0,0.0,0.0,199.0,14156.0,...,85.722468,183.024284,49.528142,0.0,0.0,27.039240,0.00000,0.0,0.000000,23.204440
Worcester,9189.0,6430.0,5289.0,0.0,0.0,0.0,0.0,0.0,329.0,6578.0,...,41.652159,168.613099,178.763967,0.0,0.0,0.000000,0.00000,0.0,2.234793,2.540330
Worcestershire,79946.0,30632.0,18139.0,0.0,0.0,13196.0,0.0,0.0,3273.0,73205.0,...,659.827173,231.455790,373.485167,0.0,0.0,193.974663,0.00000,0.0,0.000000,163.189668


NL, 10/07/22 -- to do: run 2 separate groupby's on the dataset above, one for sum, one for percent, merge the two into one DF. Then: do some actual analysis. 